# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [34]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/AI/Project/6th/Sequential NLP/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

10# Load the dataset [10 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd

In [0]:
df1 = pd.read_csv(project_path+'/train_bodies.csv')
df2 = pd.read_csv(project_path+'/train_stances.csv')

In [0]:
dataset = pd.merge(df1,df2,on='Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [40]:
dataset.head()

Body ID  ...     Stance
0        0  ...  unrelated
1        0  ...  unrelated
2        0  ...  unrelated
3        0  ...  unrelated
4        0  ...  unrelated

[5 rows x 4 columns]

## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [42]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [10 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
%tensorflow_version 2.x
from tensorflow.keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
custom_tokenizer = Tokenizer(num_words = MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [47]:
custom_tokenizer.fit_on_texts(dataset['articleBody'])
custom_tokenizer.fit_on_texts(dataset['Headline'])
custom_tokenizer.document_count

99944

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = []
articles = []
from nltk import sent_tokenize
for i in range(len(dataset['articleBody'])):
  text = dataset['articleBody'][i]
  texts.append(text)
  sentences = sent_tokenize(texts[i])
  articles.append(sentences)

## Check 2:

first element of texts and articles should be as given below. 

In [50]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [51]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [10 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

## for article body
data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
## word index encoding
for i, sentences in enumerate(articles):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                try:
                    if k < MAX_SENT_LENGTH and custom_tokenizer.word_index[word] < MAX_NB_WORDS:
                      data[i, j, k] = custom_tokenizer.word_index[word]
                      k = k + 1
                except:
#                       print(word)
                        pass


### Check 3:

Accessing first element in data should give something like given below.

In [53]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

10# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [10 marks] 

In [0]:
texts_heading = []
articles_heading = []
from nltk import sent_tokenize
for i in range(len(dataset['Headline'])):
  text_h = dataset['Headline'][i]
  texts_heading.append(text_h)
  sentences = sent_tokenize(texts_heading[i])
  articles_heading.append(sentences)

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

data_heading = np.zeros((len(texts_heading), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
## word index encoding
for i, sentences in enumerate(articles_heading):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTS:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                try:
                    if k < MAX_SENT_LENGTH and custom_tokenizer.word_index[word] < MAX_NB_WORDS:
                      data_heading[i, j, k] = custom_tokenizer.word_index[word]
                      k = k + 1
                except:
#                       print(word)
                        pass


### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
from tensorflow.keras.utils import to_categorical

In [54]:
from sklearn.preprocessing import LabelEncoder
lenc = LabelEncoder()
dataset.Stance1 = lenc.fit_transform(dataset.Stance)
labels = to_categorical(dataset.Stance1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
labels.shape

(49972, 4)

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [58]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train,x_val,y_train,y_val = train_test_split(data,labels,train_size=0.8)
x_heading_train,x_heading_val = train_test_split(data_heading,train_size=0.8)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [55]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [56]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
vocab_size = len(custom_tokenizer.word_index) + 1
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in custom_tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [20 marks]  

### Import layers from Keras to build the model

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed

### Model

In [0]:
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedded_sequences = Embedding(output_dim=100, input_dim=vocab_size, input_length=MAX_SENT_LENGTH, weights=[embedding_matrix])(sentence_input)
x = LSTM(400)(embedded_sequences)
x = Dense(64,activation='relu')(x)
preds = Dense(4,activation='sigmoid')(x)
model = Model(inputs=sentence_input,outputs=preds)

### Compile and fit the model

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [66]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 100)           2787400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 400)               801600    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                25664     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 3,614,924
Trainable params: 3,614,924
Non-trainable params: 0
_________________________________________________________________


In [67]:
new_history = model.fit(x_train.reshape(x_train.shape[0],400),y_train,validation_data=(x_val.reshape(x_val.shape[0],400),y_val),epochs=10,batch_size=512,verbose=1)

Epoch 1/10
79/79 [==============================] - 21s 270ms/step - loss: 0.8217 - accuracy: 0.7270 - val_loss: 0.8025 - val_accuracy: 0.7252
Epoch 2/10
79/79 [==============================] - 21s 265ms/step - loss: 0.7870 - accuracy: 0.7328 - val_loss: 0.7960 - val_accuracy: 0.7252
Epoch 3/10
79/79 [==============================] - 21s 265ms/step - loss: 0.7745 - accuracy: 0.7328 - val_loss: 0.7878 - val_accuracy: 0.7252
Epoch 4/10
79/79 [==============================] - 21s 265ms/step - loss: 0.7628 - accuracy: 0.7343 - val_loss: 0.7688 - val_accuracy: 0.7308
Epoch 5/10
79/79 [==============================] - 21s 264ms/step - loss: 0.7480 - accuracy: 0.7434 - val_loss: 0.7518 - val_accuracy: 0.7399
Epoch 6/10
79/79 [==============================] - 21s 265ms/step - loss: 0.7065 - accuracy: 0.7517 - val_loss: 0.6991 - val_accuracy: 0.7515
Epoch 7/10
79/79 [==============================] - 21s 264ms/step - loss: 0.6658 - accuracy: 0.7630 - val_loss: 0.6758 - val_accuracy: 0.7583

In [68]:
import matplotlib.pyplot as plt

fig,(ax1,ax2) = plt.subplots(1,2,figsize=(14,6))
fig.suptitle("Model stats")
fig.subplots_adjust(top=0.85, wspace=0.3)

ax1.plot(new_history.history['accuracy'],label='Train Accuracy')
ax1.plot(new_history.history['val_accuracy'],label='Validation Accuracy')
ax1.set_title('Accuracy')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Accuracy')
ax1.legend(loc='best')

ax2.plot(new_history.history['loss'],label='Train Loss')
ax2.plot(new_history.history['val_loss'],label='Test Loss')
ax2.set_title('Loss')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Loss')
ax2.legend(loc='best')

## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)